# Starbucks data analysis(crawling)

In [ ]:
# 스타벅스 데이터 분석(Crawling)
# 스타벅스 홈페이지 >> 전국 매장 정보(지도) >> 크롤링
# 서울시 열린 데이터 광장 >> Open API(Application Programming Interface) >> 서울시 데이터 활용
# Open API(Application Programming Interface): 데이터 분석을 위해 데이터를 제공할 수 있도록 만든 인터페이스

# 라이브러리 추가 설치: xlrd / openpyxl

In [19]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

browser = webdriver.Chrome("C:/mychrome/chromedriver.exe")
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
browser.get(url)

# 서울시 버튼
seoulBtn = "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a"
browser.find_element_by_css_selector(seoulBtn).click()
time.sleep(2)

# 전체 버튼
allBtn = "#mCSB_2_container > ul > li:nth-child(1) > a"
browser.find_element_by_css_selector(allBtn).click()
time.sleep(2)


In [76]:
# 현재 떠있는 html 파일을 가지고와서 읽어야해.
# 매장 리스트 정보(tag, 규칙 확인) 가져오기.
# 

html = browser.page_source  # browser의 소스코드를 모두 html에 저장.
soup = BeautifulSoup(html,'html.parser') 
starbucksSoupList = soup.select('li.quickResultLstCon')  # 모두 tag 이름이 같음.



In [71]:
# starbucksStore = starbucksSoupList[0]
# name = starbucksStore.select('strong')[0].getText('',1)
# lat = starbucksStore['data-lat'].strip()
# lng = starbucksStore['data-long'].strip()

# # starbucksStore.select('i')  # [<i class="pin_general">리저브 매장 2번</i>]
# # starbucksStore.select('i')[0]['class']  # ['pin_general']
# # starbucksStore.select('i')[0]['class'][0]  # 'pin_general'
# storeType = starbucksStore.select('i')[0]['class'][0][4:]  # 매장타입: general

# # starbucksStore.select('p')[0].getText('',1).split()  # split: 구분자를 지정하지 않으면 공백으로 자동 구분
# address = str(starbucksStore.select('p')[0]).split('<br/>')[0].split('>')[1]
# tel = str(starbucksStore.select('p')[0]).split('<br/>')[1].split('<')[0]


#### 매장 리스트 정보 추출

In [80]:
starbucksList = []

for i in starbucksSoupList:
    name = i.select('strong')[0].getText('',1)
    lat = i['data-lat'].strip()
    lng = i['data-long'].strip()
    storeType = i.select('i')[0]['class'][0][4:]
    address = str(i.select('p')[0]).split('<br/>')[0].split('>')[1]
    tel = str(i.select('p')[0]).split('<br/>')[1].split('<')[0]
    starbucksList.append([name, lat, lng, storeType, address, tel])


## DataFrame

In [84]:
columns = ['매장명','위도','경도','매장타입','주소','전화번호']

seoulStarbucksDf = pd.DataFrame(starbucksList, columns=columns)
# seoulStarbucksDf.head()  # 상위 5개 정보 출력  /  tail(): 하위 5개 정보 출력
# seoulStarbucksDf.info()  # null: 비어있음  /  non-null: 비어있지않아.
# Pandas에서 object는 string과 같다.

# dataframe >> excel

seoulStarbucksDf.to_excel("seoul_starbucks_list.xlsx",index=False)  # 엑셀로 출력 / 순번 삭제


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     522 non-null    object
 1   위도      522 non-null    object
 2   경도      522 non-null    object
 3   매장타입    522 non-null    object
 4   주소      522 non-null    object
 5   전화번호    522 non-null    object
dtypes: object(6)
memory usage: 24.6+ KB


# 서울시 인구 통계 불러오기

In [128]:
# My 인증키 
SEOUL_API_AUTH_KEY = '6553496e76646e6a3831755646475a'

# API 호출을 위한 URL 작성
url = 'http://openapi.seoul.go.kr:8088/{}/json/SdeTlSccoSigW/1/25/'.format(SEOUL_API_AUTH_KEY)  # 시작페이지 1 / 마지막페이지 25

import requests

result_dict = requests.get(url).json()  # 클라이언트 요청에 의해 정상적으로 반응함 <Response [200]>  //  Error가 4xx >> 주소가 잘못된 것  // Error 5xx : Internal Error >> 서버 에러
# json파일의 형식이 dictionary!!
result_data = result_dict['SdeTlSccoSigW']
result_data['row'][0]  # 도봉구 정보

data_list = result_data['row']
data_list  # 딕셔너리가 요소로 들어가있는 길이가 25인 리스트
sampleDf = pd.DataFrame(data_list)  # 딕셔너리를 dataframe으로 넣으면 Key가 1행에서 분류하는 행으로 변환
result_dict

{'SdeTlSccoSigW': {'list_total_count': 25,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
  'row': [{'OBJECTID': 1.0,
    'SIG_CD': '11320',
    'SIG_KOR_NM': '도봉구',
    'SIG_ENG_NM': 'Dobong-gu',
    'ESRI_PK': 0.0,
    'LAT': '37.6658609',
    'LNG': '127.0317674'},
   {'OBJECTID': 2.0,
    'SIG_CD': '11380',
    'SIG_KOR_NM': '은평구',
    'SIG_ENG_NM': 'Eunpyeong-gu',
    'ESRI_PK': 1.0,
    'LAT': '37.6176125',
    'LNG': '126.9227004'},
   {'OBJECTID': 3.0,
    'SIG_CD': '11230',
    'SIG_KOR_NM': '동대문구',
    'SIG_ENG_NM': 'Dongdaemun-gu',
    'ESRI_PK': 2.0,
    'LAT': '37.5838012',
    'LNG': '127.0507003'},
   {'OBJECTID': 4.0,
    'SIG_CD': '11590',
    'SIG_KOR_NM': '동작구',
    'SIG_ENG_NM': 'Dongjak-gu',
    'ESRI_PK': 3.0,
    'LAT': '37.4965037',
    'LNG': '126.9443073'},
   {'OBJECTID': 5.0,
    'SIG_CD': '11545',
    'SIG_KOR_NM': '금천구',
    'SIG_ENG_NM': 'Geumcheon-gu',
    'ESRI_PK': 4.0,
    'LAT': '37.4600969',
    'LNG': '126.9001546'},
   {'OBJECTID': 6.0

In [137]:
# 서울시 구 단위 정보를 추출하는 서비스

def seoul_open_api_data(url, service):
    data_list = None
    try:
        result_dict = requests.get(url).json()
        result_data = result_dict[service]
        code = result_data['RESULT']['CODE']
        if code == 'INFO-000':
            data_list = result_data['row']
    except:
        pass
    return data_list

seoul_data_list = seoul_open_api_data(url, 'SdeTlSccoSigW')  # 서비스이름: SdeTlSccoSigW 등

if seoul_data_list == None:
    print('key Error')

else:
    print(seoul_data_list)



[{'OBJECTID': 1.0, 'SIG_CD': '11320', 'SIG_KOR_NM': '도봉구', 'SIG_ENG_NM': 'Dobong-gu', 'ESRI_PK': 0.0, 'LAT': '37.6658609', 'LNG': '127.0317674'}, {'OBJECTID': 2.0, 'SIG_CD': '11380', 'SIG_KOR_NM': '은평구', 'SIG_ENG_NM': 'Eunpyeong-gu', 'ESRI_PK': 1.0, 'LAT': '37.6176125', 'LNG': '126.9227004'}, {'OBJECTID': 3.0, 'SIG_CD': '11230', 'SIG_KOR_NM': '동대문구', 'SIG_ENG_NM': 'Dongdaemun-gu', 'ESRI_PK': 2.0, 'LAT': '37.5838012', 'LNG': '127.0507003'}, {'OBJECTID': 4.0, 'SIG_CD': '11590', 'SIG_KOR_NM': '동작구', 'SIG_ENG_NM': 'Dongjak-gu', 'ESRI_PK': 3.0, 'LAT': '37.4965037', 'LNG': '126.9443073'}, {'OBJECTID': 5.0, 'SIG_CD': '11545', 'SIG_KOR_NM': '금천구', 'SIG_ENG_NM': 'Geumcheon-gu', 'ESRI_PK': 4.0, 'LAT': '37.4600969', 'LNG': '126.9001546'}, {'OBJECTID': 6.0, 'SIG_CD': '11530', 'SIG_KOR_NM': '구로구', 'SIG_ENG_NM': 'Guro-gu', 'ESRI_PK': 5.0, 'LAT': '37.4954856', 'LNG': '126.858121'}, {'OBJECTID': 7.0, 'SIG_CD': '11110', 'SIG_KOR_NM': '종로구', 'SIG_ENG_NM': 'Jongno-gu', 'ESRI_PK': 6.0, 'LAT': '37.5990998'

In [138]:
# 일반화 함수를 이용하여 시군구 open api 호출
SEOUL_API_AUTH_KEY = '6553496e76646e6a3831755646475a'
# API 호출을 위한 URL 작성
sgg_url = 'http://openapi.seoul.go.kr:8088/{}/json/SdeTlSccoSigW/1/25/'.format(SEOUL_API_AUTH_KEY)
sgg_data_list = seoul_open_api_data(sgg_url, 'SdeTlSccoSigW')

sggDf = pd.DataFrame(data = sgg_data_list, columns=['SIG_CD','SIG_KOR_NM','LAT','LNG'])
sggDf.columns = ['시군구코드','시군구명','위도','경도']
sggDf.to_excel("seoul_sgg_list.xlsx", index=0)

In [142]:
# 일반화 함수를 이용하여 시군구 open api 호출
SEOUL_API_AUTH_KEY = '6553496e76646e6a3831755646475a'

pop_url = 'http://openapi.seoul.go.kr:8088/{}/json/octastatapi419/1/26/'.format(SEOUL_API_AUTH_KEY)
pop_data_list = seoul_open_api_data(pop_url, 'octastatapi419')
sgg_pop_df = pd.DataFrame(pop_data_list)
sgg_pop_df.head()
sgg_pop_df.info()





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   GIGAN                  26 non-null     object
 1   JACHIGU                26 non-null     object
 2   SEDAE                  26 non-null     object
 3   GYE_1                  26 non-null     object
 4   NAMJA_1                26 non-null     object
 5   YEOJA_1                26 non-null     object
 6   GYE_2                  26 non-null     object
 7   NAMJA_2                26 non-null     object
 8   YEOJA_2                26 non-null     object
 9   GYE_3                  26 non-null     object
 10  NAMJA_3                26 non-null     object
 11  YEOJA_3                26 non-null     object
 12  SEDAEDANGINGU          26 non-null     object
 13  N_65SEISANGGORYEONGJA  26 non-null     object
dtypes: object(14)
memory usage: 3.0+ KB


In [169]:
sgg_pop_df_selected = sgg_pop_df[sgg_pop_df['JACHIGU'] != '합계']  # 자치구가 '합계'와 값이 같지 않냐?
sgg_pop_df_final = sgg_pop_df_selected[['JACHIGU','GYE_1']]  # 1개 추출: 대괄호 1 or 2쌍  /  2개 추출: 대괄호 2쌍
  # boolean 참조라고 한다!! 
# >> 합계를 제외한 나머지 값 추출
sgg_pop_df_final.columns=['시군구명','주민등록인구']
sgg_pop_df_final.to_excel('sgg_pop.xlsx',index=0)

# 서울시 동 별 사업체 현황 api 호출

In [196]:
SEOUL_API_AUTH_KEY = '6553496e76646e6a3831755646475a'
biz_url = 'http://openapi.seoul.go.kr:8088/{}/json/octastatapi104/1/450/'.format(SEOUL_API_AUTH_KEY)  # 8088: 포트번호!! 물리적인 통로는 1개인데 논리적인 통로는 65536개 있음(포트가 65535번까지). 
biz_data_list = seoul_open_api_data(biz_url, 'octastatapi104')
sgg_biz_df = pd.DataFrame(biz_data_list)

# 구 단위 사업체 소계 현황
sgg_biz_df_selected = sgg_biz_df[sgg_biz_df['DONG'] == '소계']

sgg_biz_df_final = sgg_biz_df_selected[['JACHIGU','GYE','SAEOPCHESU_1']]
sgg_biz_df_final.columns = ['시군구명','종사자수','사업체수']
sgg_biz_df_final = sgg_biz_df_final.reset_index(drop = 1)
sgg_biz_df_final.to_excel("sgg_biz.xlsx",index=0)


In [222]:
# 엑셀파일 불러오기(Pandas 사용)

seoul_starbucks = pd.read_excel("seoul_starbucks_list.xlsx")

seoul_starbucks.head()

,매장명,위도,경도,매장타입,주소,전화번호,시군구명
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232,강남구
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232,강남구
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,강남구
4,스탈릿대치R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,강남구


In [223]:
sgg_names = []
for addr in seoul_starbucks['주소']:
    sgg_names.append(addr.split()[1])

seoul_starbucks['시군구명'] = sgg_names
seoul_starbucks.to_excel('seoul_starbucks_list.xlsx', index = False)

# 데이터 병합

In [246]:
# 보고 싶은 데이터: 구 별로 스타벅스 매장 갯수
#                 스타벅스 매장 수
# 시군구명
# 강남구              200
# 금천구              29
# ...                 ...
# 피봇(pivot)함수: 피봇팅(특정 기준에 따른 데이터 요약)
# pandas는 2가지 함수 있음 pivot table, pivot

seoul_sgg = pd.read_excel('seoul_sgg_list.xlsx')
# >> 시군구코드	  시군구명  	위도	경도

seoul_starbucks = pd.read_excel('seoul_starbucks_list.xlsx')
# >> 매장명	 위도	경도	매장타입	주소	전화번호	시군구명

seoul_sgg = pd.read_excel('seoul_sgg_list.xlsx')
# >> 시군구코드	  시군구명  	위도	경도

seoul_starbucks = pd.read_excel('seoul_starbucks_list.xlsx')
# >> 매장명	 위도	경도	매장타입	주소	전화번호	시군구명

starbucks_sgg_count = seoul_starbucks.pivot_table(index = '시군구명', values = '매장명', aggfunc = 'count').rename(columns = {'매장명': '스타벅스_매장수'})  # 매장명 >> 스타벅스_매장수로 변경
starbucks_sgg_count = starbucks_sgg_count.drop(['67'])  # 특정 행 삭제 drop!
starbucks_sgg_count

# 병합
seoul_sgg = pd.merge(seoul_sgg, starbucks_sgg_count, how = 'left', on = '시군구명')  # 왼쪽이 기준이 되어 합치기(merge)
seoul_sgg_pop = pd.read_excel('sgg_pop.xlsx')
seoul_sgg = pd.merge(seoul_sgg, seoul_sgg_pop, how = 'left', on = '시군구명')
seoul_sgg_biz = pd.read_excel('sgg_biz.xlsx')
seoul_sgg = pd.merge(seoul_sgg,seoul_sgg_biz, how = 'left', on = '시군구명')
seoul_sgg.head()



,시군구코드,시군구명,위도,경도,스타벅스_매장수,주민등록인구,종사자수,사업체수
0,11320,도봉구,37.665861,127.031767,2,"371,095",68669,18455
1,11380,은평구,37.617612,126.922700,8,"474,165",87693,24179
2,11230,동대문구,37.583801,127.050700,8,"374,039",143858,32994
3,11590,동작구,37.496504,126.944307,11,"407,966",103915,19609
4,11545,금천구,37.460097,126.900155,10,"262,337",223058,30080


In [247]:
# 병합 결과를 파일로 저장
seoul_sgg.to_excel("seoul_sgg_stat.xlsx", index = 0)